# Atlas Vector Search - Semantic Search for Text

This notebook is a companion to the [Semantic Search for Text](https://www.mongodb.com/docs/atlas/atlas-vector-search/tutorials/vector-search-tutorial/) page. Refer to the page for set-up instructions and detailed explanations.

<a target="_blank" href="https://colab.research.google.com/github/mongodb/docs-notebooks/blob/main/run-queries/semantic-search-and-or-filter.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [ ]:
pip install pymongo

In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.operations import SearchIndexModel
import time

# Connect to your Atlas deployment
uri = "<connection-string>"
client = MongoClient(uri)

# Access your database and collection
database = client["sample_mflix"]
collection = database["embedded_movies"]

# Create your index model, then create the search index
search_index_model = SearchIndexModel(
  definition={
    "fields": [
      {
        "type": "vector",
        "path": "plot_embedding",
        "numDimensions": 1536,
        "similarity": "dotProduct",
        "quantization": "scalar"
      },
      {
        "type": "filter",
        "path": "genres"
      },
      {
        "type": "filter",
        "path": "year"
      }
    ]
  },
  name="vector_index",
  type="vectorSearch",
)

result = collection.create_search_index(model=search_index_model)
print("New search index named " + result + " is building.")

# Wait for initial sync to complete
print("Polling to check if the index is ready. This may take up to a minute.")
predicate=None
if predicate is None:
  predicate = lambda index: index.get("queryable") is True

while True:
  indices = list(collection.list_search_indexes(result))
  if len(indices) and predicate(indices[0]):
    break
  time.sleep(5)
print(result + " is ready for querying.")

## Pre-Filter by AND Operator

In [ ]:
# define pipeline
pipeline = [
  {
    '$vectorSearch': {
      'index': 'vector_index', 
        'path': 'plot_embedding', 
        'filter': {
          '$and': [
            {
              'genres': {
                '$nin': [
                  'Drama', 'Western', 'Crime'
                ], 
                '$in': [
                  'Action', 'Adventure', 'Family'
                ]
              }
            }, {
              'year': {
                '$gte': 1960, 
                '$lte': 2000
              }
            }
          ]
        }, 
        'queryVector': [-0.020156775, -0.024996493, 0.010778184, -0.030058576, -0.03309321, 0.0031229265, -0.022772837, 0.0028351594, 0.00036870153, -0.02820117, 0.016245758, 0.0036232488, 0.0020519753, -0.0076454473, 0.0073380596, -0.007377301, 0.039267123, -0.013433489, 0.01428371, -0.017279103, -0.028358135, 0.0020160044, 0.00856761, 0.009653277, 0.0107912645, -0.026683854, 0.009594415, -0.020182934, 0.018077003, -0.015709465, 0.003310956, 0.0014878864, -0.015971072, -0.002411684, -0.029561523, -0.030450987, -0.013106481, -0.005385822, -0.018652538, 0.012642129, -0.005189617, 0.018835662, -0.0048102876, -0.0261214, -0.016167276, -0.007972456, 0.0023381072, -0.010058766, -0.009012341, 0.008358325, 0.018665617, 0.02163485, -0.012975678, -0.010745483, -0.002571918, -0.014479915, 0.007226877, 0.015003128, 0.013165343, -0.028279653, 0.0053727417, -0.020588424, -0.017383745, 0.023518417, 0.01262905, -0.011922712, 0.007638907, -0.0073249796, -0.014859244, -0.00001101736, 0.017043658, 0.010111088, 0.0074623227, 0.009555174, 0.008338705, -0.002240005, -0.0010603234, -0.004973792, 0.003391073, 0.021543289, 0.013341927, 0.0005980159, 0.010693162, 0.005336771, 0.016062634, 0.005768421, 0.005186347, 0.039790336, 0.0021942237, -0.0026275094, 0.010431555, 0.0042151334, -0.0050359233, 0.025768232, -0.021451725, 0.01833861, -0.01836477, -0.013433489, 0.030006256, -0.014793842, 0.017475309, 0.0020585153, -0.012975678, -0.017266022, -0.01593183, -0.014257549, 0.0010676811, -0.007887433, -0.0045911926, 0.00012303676, -0.0014976967, 0.03552615, 0.0065630507, -0.037435878, 0.011929252, -0.00939167, 0.016768971, 0.01223664, 0.007789331, -0.037200432, 0.013145722, 0.00896002, 0.021857215, 0.010333453, 0.021582529, -0.007089534, -0.007154935, -0.02485261, 0.0040254686, -0.00088864425, 0.023466095, -0.020719228, -0.006690584, -0.021006994, -0.018286288, 0.025545865, -0.0096598165, 0.008803056, -0.023021365, -0.040078104, 0.015408617, 0.017043658, -0.011242535, 0.0063537657, -0.026618453, 0.0071614753, -0.014623798, 0.00067322777, -0.00083427917, -0.028070368, 0.03714811, -0.004529061, 0.0054087127, 0.0028727653, 0.008384486, 0.010026066, -0.006190262, -0.0002493436, 0.0029953935, -0.026226042, -0.018417092, 0.009941043, 0.0036494094, -0.00982332, 0.013551212, 0.02574207, -0.0022645304, -0.0006004685, 0.012805633, -0.024303235, 0.008194821, -0.014179068, -0.02977081, 0.003095131, -0.0015941641, 0.029953934, 0.0052680993, 0.025388902, -0.031392768, -0.021386323, 0.014898485, 0.022419669, 0.00897964, 0.013243824, 0.006854088, 0.0066415328, -0.003839074, -0.01877026, 0.021216279, -0.015055449, -0.0015508354, 0.013211124, -0.008783435, 0.0052157775, -0.68938524, -0.01221702, -0.04125533, -0.016232677, 0.020039052, -0.0026422248, -0.0037050007, 0.0064682183, -0.0047579664, 0.0032749851, -0.0035382267, 0.031942144, -0.00035643874, -0.011628405, -0.043086577, -0.0196074, -0.0066088317, -0.014872325, 0.028331975, 0.010294212, -0.013930541, 0.031994462, -0.018626377, 0.017462227, 0.026343765, -0.010274592, 0.0046827546, -0.029430721, -0.011746128, 0.0024362097, 0.0023054064, 0.0027730279, -0.002406779, 0.003917556, 0.059436977, 0.008665713, -0.0018901062, 0.06037876, 0.017880797, 0.05185039, 0.0067102043, -0.020300657, 0.005604917, 0.018704858, 0.012073136, 0.0144145135, 0.012413224, -0.0074819434, 0.015801027, -0.0061412104, 0.008613391, -0.0039077457, -0.0036232488, 0.008469507, 0.014087505, 0.0124066835, 0.019267311, -0.002573553, 0.005055544, -0.009417831, -0.009103903, 0.011150973, -0.012046975, 0.0058567133, -0.0053727417, 0.018260127, -0.005588567, 0.015591742, 0.007495024, -0.02567667, 0.024211673, 0.021386323, -0.012890656, -0.016114954, 0.009515933, 0.009679437, 0.025532786, -0.0076454473, -0.02575515, 0.008319084, -0.0068410076, -0.017082898, -0.026173722, -0.0049901423, 0.01918883, -0.008646091, -0.031759016, 0.014820003, 0.011850771, 0.01836477, 0.012700991, -0.0011437106, 0.005058814, 0.0151993325, -0.0060692686, 0.027416352, 0.0037344315, 0.0013546307, 0.018325528, -0.03152357, -0.008809595, 0.014649959, -0.008345244, 0.0066415328, -0.005523165, 0.0043492066, -0.0015892589, 0.0048855, 0.034453563, -0.03837766, 0.0068410076, -0.0042151334, -0.0067429054, 0.0055689462, -0.011733048, -0.0212032, 0.016847452, -0.0022220195, 0.0059351954, -0.00449963, 0.02251123, -0.01020265, 0.023361452, -0.0032455544, 0.016180357, 0.0049443613, -0.0064747585, -0.03259616, 0.012321662, 0.020104453, 0.009954124, -0.019411195, 0.0048102876, -0.000392614, 0.012184318, 0.0044276887, 0.005634348, -0.020562263, 0.015722545, -0.005179807, -0.0067952266, 0.0027861083, 0.0024198592, -0.0020585153, 0.0018525004, -0.045100946, -0.010176489, -0.012956058, 0.0013497255, 0.0105361985, 0.003796563, -0.0106016, -0.013126101, 0.0050359233, 0.015003128, -0.0075800456, -0.015722545, -0.01755379, -0.00978408, -0.02940456, 0.017606111, 0.016612006, -0.016912855, 0.025441224, 0.0054741143, 0.00448001, 0.009470152, 0.015382457, -0.008332164, -0.019123428, 0.024564842, 0.016860534, 0.008286383, -0.007141855, 0.006559781, 0.016625088, -0.01840401, -0.011602244, -0.00489858, -0.0073184394, -0.008809595, -0.0018459603, -0.01629808, -0.005542786, 0.0064257076, 0.010379234, 0.014663039, 0.034872133, -0.013355007, 0.027285548, 0.011654565, -0.004032009, 0.02323065, -0.02653997, -0.0009941043, 0.002946342, 0.010667001, 0.008345244, 0.018626377, 0.04821406, 0.031392768, 0.010281132, 0.026069079, 0.002735422, 0.01182461, -0.01593183, 0.006585941, -0.010071847, 0.024564842, -0.0025261368, 0.004293615, -0.0068606283, -0.0066448026, -0.0074100015, -0.0014347476, 0.021530207, -0.010418476, 0.018495573, -0.0034924455, -0.014165987, -0.004784127, -0.012472086, 0.004417878, -0.0030313642, -0.010084927, -0.010954768, 0.01508161, 0.0010047321, 0.0042347535, -0.03345946, -0.00027346043, 0.014793842, -0.019882087, 0.012772933, 0.021490967, 0.0031932332, 0.0093589695, 0.00090172456, 0.0048102876, 0.0070045115, -0.0045584915, 0.015840268, 0.024342475, -0.0091300635, 0.0039796876, 0.003796563, 0.025022654, -0.008103259, -0.025022654, 0.03021554, -0.008201361, -0.0070502926, 0.0011821339, 0.021072397, 0.004849529, -0.02495725, 0.012184318, 0.0019228071, -0.007226877, 0.020562263, 0.018861823, -0.0017593032, 0.01345965, 0.0022727058, 0.003023189, -0.026971621, -0.0030558899, 0.017723834, -0.01998673, -0.010608139, 0.011491061, -0.025179617, 0.0069652707, 0.003924096, 0.021177039, 0.0045650317, -0.0009973744, 0.007586586, -0.004032009, -0.008129419, -0.010091467, -0.04279881, 0.019790525, 0.01595799, 0.0044309585, -0.0033747226, -0.018665617, -0.012818714, -0.016206518, 0.014113666, -0.0020912162, 0.01427063, -0.020248337, -0.0112752365, -0.020588424, -0.011039791, 0.008744194, -0.015147011, 0.0022269245, -0.010438096, -0.0017772885, -0.028750544, -0.008861917, -0.016991336, 0.033668745, 0.034636687, 0.009888723, 0.0023953337, 0.006991431, -0.003346927, 0.003103306, -0.0044571194, 0.011249076, 0.0033779927, 0.00012446742, -0.0027027212, -0.025859794, -0.011942333, 0.02694546, 0.028227331, 0.0064289775, -0.03385187, -0.020719228, 0.00489531, 0.10663077, 0.041752383, -0.021700252, -0.008103259, 0.0049574412, -0.01675589, -0.020182934, -0.006585941, 0.007684688, -0.002859685, 0.027023941, 0.00856107, 0.0037017306, 0.016978256, 0.025885954, -0.010372694, 0.0025964435, 0.011706887, 0.021360163, -0.021674091, -0.024983412, 0.0034074234, 0.0032030435, 0.022262705, -0.01266829, -0.002249815, 0.032779284, -0.0034303141, -0.016101874, -0.005156916, -0.0212032, 0.005362931, 0.009077743, -0.013917461, -0.0017315074, 0.010980929, -0.019450437, 0.013865139, 0.028227331, -0.008757275, -0.0033649125, -0.012857955, 0.011039791, 0.009764459, 0.00029594224, -0.026317604, 0.025048813, 0.037749805, -0.025807472, -0.005425063, 0.021791814, -0.010012985, -0.00066995766, -0.016952096, 0.0031147513, -0.016598927, 0.0084368065, 0.004787397, -0.0064355177, 0.0015164997, -0.021216279, -0.023845425, 0.013969782, -0.011255615, 0.0042576445, -0.024250913, -0.009908343, -0.02289056, -0.023361452, -0.010987469, -0.013394248, 0.0032553647, -0.019018786, 0.021438645, 0.029587684, -0.010490417, 0.01263559, -0.018417092, -0.008731114, 0.01875718, -0.0072399573, -0.029090632, -0.017736914, -0.04031355, -0.019712042, 0.012772933, -0.030320182, -0.022341188, -0.02041838, 0.011752668, 0.028829027, -0.017043658, 0.024996493, 0.006334145, -0.0024263994, -0.0077370093, 0.017802317, 0.017396826, 0.030398665, 0.011464901, 0.03016322, -0.014558396, -0.0036690298, -0.009954124, -0.006703664, -0.00035705187, -0.014519156, 0.0075342646, -0.00896656, 0.040078104, 0.024420958, -0.016886694, -0.00092543266, -0.0017494928, 0.01672973, 0.016533526, 0.002648765, 0.0187441, -0.0055460557, 0.004735076, 0.03186366, 0.0003435628, 0.007495024, 0.023453014, -0.012504786, -0.0074557825, -0.0027844731, -0.04570264, 0.010477337, 0.0030101088, -0.015670223, 0.03351178, -0.020261416, 0.00050849747, -0.009653277, -0.023466095, -0.007396921, -0.011909632, 0.003436854, -0.02979697, -0.039031677, -0.014584557, 0.0019555078, 0.0042216736, -0.0060594585, -0.023400694, -0.00023462824, -0.017763074, -0.016180357, 0.0132372845, -0.020496862, -0.007390381, -0.0058697937, -0.0096598165, 0.0039796876, -0.019306554, -0.012622509, -0.0012287326, 0.010863206, 0.024368636, 0.027730279, 0.016795132, 0.019908248, -0.006343955, 0.0014592733, -0.005425063, 0.019450437, 0.004532331, -0.031889822, 0.008476048, 0.019712042, -0.00047906674, -0.0028286192, 0.011883471, -0.012426305, 0.0041497317, 0.001756033, -0.0013603533, -0.008031317, -0.010281132, -0.0071222344, -0.026330685, -0.007920134, -0.026866978, -0.03026786, -0.0015328501, 0.027442513, -0.005922115, 0.005186347, 0.003436854, 0.036703378, -0.0053204205, 0.013165343, 0.0016939015, -0.0041431915, -0.017213702, -0.012439385, -0.015212413, 0.014532236, 0.0093589695, -0.0053400407, 0.017422987, -0.028881347, -0.014179068, 0.011307937, 0.040104263, -0.007593126, -0.000631943, -0.0003404971, -0.0055198953, -0.00063030794, -0.004852799, -0.0024214943, -0.029718488, 0.023322212, 0.011079031, 0.012988758, 0.0071614753, -0.034034993, -0.01551326, 0.004012388, 0.006442058, 0.032386873, 0.0076519875, 0.0465921, 0.01757995, -0.0135381315, -0.016978256, 0.024983412, 0.0003280299, 0.0026209692, 0.022380428, -0.010640841, 0.0027648527, -0.007959375, -0.005922115, 0.0075342646, -0.03597088, -0.018874902, 0.03510758, -0.015356296, 0.004597733, -0.0015328501, -0.019947488, -0.013446569, 0.020614585, -0.0056016473, 0.035186063, 0.0005248479, -0.030712591, -0.019136509, 0.004202053, -0.010339993, 0.014754602, 0.0072922786, -0.015460939, 0.027494833, -0.02974465, -0.0033616424, 0.0105819795, -0.028881347, 0.01720062, -0.0073707607, 0.0054479535, -0.0019522378, -0.018103164, -0.009110443, -0.024630243, 0.005624538, 0.01879642, -0.019345794, -0.0027681228, -0.015971072, 0.022354268, -0.0038194535, 0.018901063, -0.017357586, -0.02493109, 0.006703664, -0.0021173768, -0.005667049, -0.004535601, -0.016441964, 0.0034172337, -0.02447328, -0.003310956, -0.02078463, -0.011589164, 0.013263445, -0.014728441, -0.0187441, -0.019476596, 0.013224204, 0.015238573, -0.012380524, 0.00019058435, 0.010778184, 0.025022654, -0.036127847, 0.01470228, -0.007671608, 0.032857765, 0.002982313, 0.009829861, 0.0072203367, -0.0028237142, 0.025990596, -0.029012151, 0.0016955365, 0.012033895, -0.0049901423, -0.013629694, 0.0072464976, 0.0012704261, 0.0018868363, 0.017043658, 0.00448001, -0.009555174, -0.016520444, 0.02570283, -0.00939167, 0.01998673, 0.002001289, -0.023662299, 0.0041072206, -0.024839528, -0.007396921, -0.0034793653, -0.032020625, -0.0036003583, -0.010719323, 0.022995204, -0.01757995, -0.0043851775, -0.023884665, -0.018430172, -0.009018881, 0.00091562246, -0.0055689462, -0.012537487, 0.016455043, 0.03264848, 0.018560974, 0.014623798, 0.0025555675, -0.0060986993, 0.0058272826, -0.008462967, -0.012720612, -0.0042576445, -0.027207067, 0.014152907, -0.0029610575, 0.010241891, -0.011222915, -0.01140604, -0.022197304, -0.003433584, -0.0056899395, 0.004372097, 0.061896075, -0.005846903, -0.011863851, 0.004535601, -0.0074819434, 0.016847452, -0.0012647035, 0.021085477, 0.02409395, -0.030137058, -0.0012197399, 0.009607496, -0.008220982, -0.007893973, -0.007893973, 0.007972456, 0.010012985, 0.009143144, 0.0044734697, 0.015264734, -0.0032520946, 0.002208939, 0.011968493, -0.0012998568, -0.0114322, -0.056454662, -0.013217663, 0.0017593032, -0.00244275, -0.021399405, -0.010732403, 0.00694565, 0.0033207664, 0.0025539326, 0.01102671, -0.012589809, 0.010706242, -0.012413224, 0.01427063, -0.000049970913, -0.0056016473, 0.027965724, 0.018652538, -0.009535554, 0.0068867886, 0.004699105, -0.001245083, -0.009071202, -0.0032946058, -0.03756668, 0.034453563, -0.00408106, 0.013361547, -0.0065107294, 0.009300108, -0.016415803, 0.0059973267, -0.017422987, 0.0048822295, 0.022158062, -0.025611266, 0.01022227, -0.0061771814, -0.014218308, -0.00044636594, -0.019110348, -0.013747416, -0.013629694, -0.021896457, -0.0051634563, -0.020509942, -0.018731019, 0.0043328563, -0.032386873, -0.023086766, 0.0196074, 0.20614585, -0.014649959, -0.009712138, 0.01345965, -0.010928608, 0.0196074, 0.015814107, 0.017383745, -0.0024656404, 0.021399405, 0.013668935, -0.0063864663, -0.0015303975, -0.0012924991, -0.0030575248, -0.015539421, -0.009692517, -0.012190859, -0.02287748, 0.002936532, 0.00069325697, 0.013158802, -0.0070110518, -0.013629694, 0.01585335, -0.019829765, 0.013747416, 0.016036473, 0.011693806, 0.0071483953, -0.010156869, -0.013799738, -0.00034703725, -0.010706242, -0.02289056, 0.0039339066, -0.0015835363, -0.014532236, 0.012445925, -0.00009779583, 0.0053335004, 0.0055329753, -0.005281179, -0.007475403, 0.00040385488, -0.012942977, -0.015277814, 0.012956058, 0.00006162057, 0.007056833, -0.02571591, -0.018731019, -0.0061771814, 0.034427404, 0.0010570535, 0.0079528345, 0.024172433, 0.021386323, -0.019803606, -0.006821387, -0.011262156, 0.026605371, -0.0036951904, -0.008207901, -0.019698963, 0.042981934, -0.026212962, 0.00856761, 0.015173172, 0.0024149541, -0.0008036222, -0.005752071, -0.02898599, -0.008443347, -0.0064224373, -0.014479915, 0.036467932, -0.00086820626, 0.026396086, 0.002001289, -0.0074361623, -0.0086918725, -0.007835112, 0.021464806, 0.0008984545, -0.02489185, 0.019515838, 0.026644614, -0.0137212565, 0.00448982, 0.004211863, -0.022380428, -0.014100585, -0.01629808, 0.0074884836, 0.02652689, 0.011634945, 0.049626734, -0.023583818, -0.0021958589, -0.015735626, 0.02733787, 0.0036428692, -0.031261966, -0.012674831, 0.006196802, -0.009535554, 0.016886694, 0.010771644, -0.021490967, 0.014100585, -0.007063373, 0.00043778197, -0.012151618, -0.0058894143, 0.009182385, -0.005768421, -0.013995943, 0.004725266, -0.01347273, -0.020797709, -0.018037762, 0.020274498, 0.011595704, 0.0017364125, -0.02248507, 0.005954816, 0.0062196925, -0.014257549, -0.025127295, 0.015356296, 0.005179807, 0.021726413, -0.0034499345, -0.017082898, 0.019803606, 0.005209238, 0.0005939283, -0.0035807376, -0.011661106, 0.006559781, 0.0033207664, 0.0017233322, -0.00059924216, -0.000341519, -0.0140221035, 0.00084286317, -0.003306051, -0.005634348, -0.00816212, -0.009319728, -0.024447119, -0.014950806, -0.024564842, 0.0137212565, -0.010084927, 0.000044886958, -0.0033943432, 0.0025359471, 0.012478625, -0.023086766, 0.014519156, 0.020876192, -0.023282971, -0.0030804155, -0.014545316, -0.16805595, 0.01262905, 0.020719228, -0.012413224, 0.026592292, -0.0024198592, 0.041072205, 0.002658575, -0.013708176, -0.0068867886, -0.0018639456, 0.000031627806, -0.043452825, -0.028018046, -0.0105819795, 0.01266829, -0.009450532, 0.008292923, 0.0058534434, -0.006782146, 0.032229908, 0.0005955633, -0.0023103117, 0.003140912, 0.00037687673, -0.0049247406, -0.008070557, 0.017279103, -0.012759852, -0.011608784, -0.019450437, 0.016167276, 0.02248507, 0.030529467, 0.015905669, 0.0061150496, -0.016834373, 0.017344505, 0.006667693, -0.005461034, 0.0066742334, 0.01998673, 0.024591003, -0.007717389, 0.0096598165, 0.03225607, 0.018626377, -0.020248337, 0.0017740185, 0.012589809, 0.0014927916, -0.040235065, 0.01713522, 0.016206518, 0.017776156, 0.024734886, 0.0040516295, -0.009627116, 0.002001289, -0.010496957, -0.0121058365, -0.017266022, 0.008279843, -0.02122936, -0.01349889, -0.02251123, 0.004820098, -0.000071533, -0.022628954, 0.015238573, -0.01833861, -0.016572766, -0.0031523572, -0.008064018, 0.019973649, 0.0089207785, -0.03228223, 0.0040647094, -0.004784127, -0.0017920039, -0.0013775212, 0.047246117, 0.0030804155, -0.010660461, 0.02982313, 0.006088889, -0.019371955, -0.024447119, -0.011687267, -0.013708176, 0.017187541, -0.018286288, 0.019267311, 0.0011960318, 0.0046271635, 0.016886694, 0.0069129495, 0.00029062838, 0.013629694, -0.016494283, -0.017069818, 0.0058240127, 0.013943622, 0.001675916, 0.01347273, 0.023335291, 0.008129419, 0.0047187256, 0.032099105, 0.0007701039, 0.0068344674, 0.0004672127, -0.00610851, 0.026396086, -0.010738943, 0.024591003, 0.008220982, -0.019908248, 0.024682565, -0.009404751, 0.0594893, -0.009731758, -0.022628954, 0.013865139, -0.016049553, 0.0033371167, -0.107572556, -0.022341188, 0.008050937, -0.0089731, 0.004983602, 0.010771644, -0.013034539, -0.013368088, -0.0071287747, 0.0091758445, -0.017409906, -0.022118822, -0.011170594, -0.010908987, 0.050490037, 0.014584557, 0.018312449, 0.0014968792, -0.0057161, 0.024342475, -0.02699778, 0.020091372, -0.00094587065, -0.021347083, -0.003711541, 0.0016677409, -0.030738752, 0.040208906, 0.008109799, -0.017527629, -0.0009058122, 0.017776156, 0.0052779093, -0.0046206233, 0.0067952266, -0.01226934, -0.009162764, -0.01595799, 0.021582529, -0.027390191, -0.00011210243, -0.003145817, 0.01672973, -0.009999905, 0.003832534, -0.01793312, -0.0004868332, 0.027573315, 0.001756033, -0.012112376, -0.009718678, 0.0025473924, -0.027547155, -0.019084187, 0.010693162, 0.025558947, -0.02168717, -0.0068802484, -0.010869746, -0.028698223, -0.0051634563, -0.012131997, -0.014963887, 0.022210384, 0.01510777, -0.0026504, -0.013577373, 0.0058599836, 0.011281776, -0.0009393305, -0.00204053, 0.030110897, -0.029326078, 0.006491109, -0.01671665, 0.0006049648, -0.024342475, -0.008325624, 0.03722659, -0.007710849, -0.0055656764, -0.02043146, -0.015317055, -0.015212413, 0.002815539, 0.022262705, 0.00818828, 0.021778734, -0.0037409717, -0.02485261, 0.0033779927, 0.013217663, -0.0059319255, -0.018940303, 0.02409395, 0.015761785, -0.009672897, 0.011301396, -0.011582624, 0.0029725027, -0.015343216, -0.00735114, -0.075761214, 0.016821291, 0.0028040938, 0.0017233322, 0.01595799, -0.0054741143, -0.007096074, -0.011641486, -0.003554577, 0.009829861, -0.037828285, 0.024983412, 0.003793293, -0.010895907, -0.011916172, -0.017893879, 0.029640006, 0.0027452323, 0.004977062, 0.0138913, 0.0132830655, 0.010725862, 0.014205228, -0.003839074, 0.020470701, 0.0048626093, -0.010967849, 0.035343025, -0.004568302, -0.007665068, 0.0040091183, -0.02367538, -0.006821387, 0.012112376, -0.0012475356, -0.02041838, -0.030869557, -0.004865879, 0.036127847, 0.019528918, 0.00087147637, 0.0016366751, -0.006072539, -0.012380524, -0.016886694, 0.0014224849, 0.0058632535, 0.0053138803, 0.024525601, -0.008227522, 0.016167276, 0.021373244, -0.019855926, -0.011602244, -0.012223559, 0.009116983, 0.00448001, 0.0027027212, 0.0112294555, -0.025048813, 0.005958086, 0.005578757, 0.012040435, -0.019528918, -0.008096718, -0.023439934, 0.00047497914, 0.0073315194, 0.025061894, -0.016455043, 0.003992768, 0.002038895, -0.0003484679, 0.004444039, -0.014846164, 0.0018263398, 0.017305264, -0.0047154557, -0.006729825, 0.011288317, -0.009764459, -0.03220375, -0.015369376, 0.009594415, 0.031078842, 0.020967754, -0.007802411, 0.022354268, -0.010778184, 0.01833861, 0.004581382, 0.0072399573, 0.010673542, -0.012112376, -0.023073684, 0.0066448026, -0.027887244, 0.0063504954, 0.012956058, 0.032151427, -0.018103164, 0.0048855, -0.018286288, -0.036938824, -0.012354363, 0.020039052, 0.004921471, -0.03790677, 0.0212686, 0.02982313, 0.015434778, 0.0041039507, -0.016245758, 0.012171238, -0.006415897, 0.0072464976, -0.0024362097, -0.025218857, -0.021399405, 0.036860343, 0.0056572384, 0.017004417, 0.03432276, -0.013825899, 0.028724384, 0.008528369, 0.018652538, -0.02443404, -0.025637427, 0.006497649, -0.015447859, 0.01917575, -0.016520444, -0.008678793, -0.021072397, 0.015840268, -0.006324335, 0.025925195, -0.03594472, 0.0384823, 0.01308032, 0.0054217926, 0.00448328, -0.027207067, -0.016847452, 0.0036003583, 0.01061468, -0.019816685, -0.004659864, 0.023387613, -0.005461034, 0.004326316, 0.0037278912, -0.007540805, 0.00860031, 0.0015524705, 0.020039052, -0.0028367946, 0.0049509015, 0.009162764, 0.009705598, 0.013982862, 0.004852799, 0.0061869915, -0.0083910255, 0.012975678, -0.034558207, -0.029064473, -0.03058179, -0.019450437, 0.01062122, -0.014179068, -0.010012985, 0.007874353, -0.014126746, -0.009731758, -0.03398267, -0.000115883464, -0.0029725027, -0.024290156, 0.012864495, -0.00937859, -0.035264544, 0.0027959184, 0.012982218, -0.012609429, 0.0065270797, 0.010712783], 
       'numCandidates': 200, 
       'limit': 10
    }
  }, {
    '$project': {
      '_id': 0, 
      'title': 1, 
      'genres': 1, 
      'plot': 1, 
      'year': 1, 
      'score': {
        '$meta': 'vectorSearchScore'
      }
    }
  }
]

# run pipeline
result = client["sample_mflix"]["embedded_movies"].aggregate(pipeline)

# print results
for i in result:
    print(i)


## Pre-Filter by OR and AND Operators

In [ ]:
# define pipeline
pipeline = [
  {
    '$vectorSearch': {
      'index': 'vector_index', 
      'path': 'plot_embedding', 
      'filter': {
        '$or': [
          {
            'genres': {
              '$ne': 'Crime'
            }
          }, {
            '$and': [
              {
                'year': {
                  '$lte': 2015
                }
              }, {
                'genres': {
                  '$eq': 'Action'
                }
              }
            ]
          }
        ]
      }, 
      'queryVector': [-0.016465975, -0.0036450154, 0.001644484, -0.028249683, -0.02077106, 0.00031438665, -0.02351539, -0.017519485, -0.0025362284, -0.038888834, 0.023489377, 0.022695992, 0.009481592, 0.009995341, -0.0043506073, -0.0021297815, 0.033972453, 0.00070193375, 0.007335553, -0.017909674, 0.015191358, 0.015360439, -0.00020505243, -0.023801528, -0.013539557, -0.0015290531, 0.009631164, -0.026493832, -0.0245689, -0.02481602, 0.02832772, -0.012473041, -0.006538917, -0.021707514, -0.01676512, -0.005865841, 0.010359517, -0.013246916, 0.014710125, 0.0060869483, 0.016192842, 0.0026484078, 0.002417546, -0.007764761, -0.0053781047, 0.0016843157, 0.009071894, 0.0026158919, -0.016465975, 0.019054228, 0.030720878, 0.028145632, -0.0124340225, -0.011491066, -0.01273967, -0.0076542073, 0.00685432, 0.002936172, 0.03194347, -0.030798918, -0.004961903, 0.012661632, -0.024646938, 0.0085256295, -0.01133499, 0.016648063, -0.017857648, 0.01627088, -0.020823086, -0.002233832, 0.008330535, 0.024894057, 0.008558145, 0.013786677, 0.027235191, -0.0075046346, -0.015308415, -0.0031442728, 0.009136925, 0.010548109, 0.009455579, -0.0070624207, -0.001147806, 0.008928824, 0.011042348, -0.019314354, -0.0035702293, 0.019691538, 0.013929747, -0.00071615935, -0.0016477356, -0.0029556816, 0.009592146, 0.011699166, -0.045183886, 0.018533977, -0.025063138, 0.002194813, -0.013968766, 0.0037848332, -0.014775156, 0.0138777215, -0.023112195, -0.019288342, -0.02252691, -0.009813253, -0.0030792414, 0.00088036386, 0.0048285886, -0.019691538, -0.021863589, 0.0068413136, 0.01641395, -0.04494977, -0.0027475806, -0.014710125, 0.0077192388, -0.0030954992, 0.005417124, -0.008219982, 0.014046803, 0.01662205, 0.027235191, 0.0015266144, 0.036261562, 0.02006872, -0.032489736, -0.0019379386, 0.004106739, -0.006808798, 0.048825648, 0.0096766865, 0.0132794315, 0.0066722315, -0.026116649, 0.019912645, -0.017441448, -0.0014128092, -0.041880284, -0.018599007, 0.018729072, 0.00619425, -0.01947043, -0.009748221, -0.012121871, 0.0001831043, 0.037198015, 0.03451872, -0.0045684627, -0.0040742233, 0.022708999, -0.028847972, 0.027287217, 0.0008214291, 0.018156795, -0.007816786, -0.017519485, 0.00501718, -0.0013811064, -0.0148401875, 0.012752676, -0.01686917, -0.0018013725, -0.014645093, -0.0064218603, 0.016426956, 0.008558145, 0.013207897, -0.012518563, -0.00018208819, 0.022643967, 0.020745048, -0.018143788, 0.0123169655, -0.018039737, 0.019626506, -0.009162938, 0.0030255904, -0.02942025, -0.007511138, 0.0050139283, -0.006408854, 0.04065769, 0.011419531, 0.000562116, 0.032775875, -0.012043833, 0.009410057, -0.012694148, -0.019041222, 0.015646579, 0.021109223, -0.010749706, -0.01915828, -0.6842354, -0.022748018, 0.010444058, -0.013090841, 0.031241132, 0.0032792133, 0.030772904, -0.0061389734, -0.018013725, -0.0103335045, 0.0075046346, 0.012395004, -0.0035442165, -0.011256952, 0.017064264, -0.00813544, 0.014319936, -0.044559583, 0.0053065703, 0.00027028716, -0.014892213, 0.012499054, 0.002056621, -0.006860823, -0.020875111, -0.0028890243, -0.01048958, 0.0043993806, -0.004360362, 0.01922331, -0.031501256, 0.025882537, 0.0032922195, -0.0058300737, 0.053117726, 0.0013900483, -0.029784426, 0.055875063, 0.02417871, 0.038888834, -0.00365477, -0.0024793257, 0.0005182197, -0.008805265, -0.007589176, 0.013643608, 0.03685985, -0.024152698, 0.00400594, -0.0148401875, 0.01971755, 0.0002887805, -0.0050594504, -0.020120746, 0.00209564, 0.00084947393, 0.004688771, -0.013526551, 0.016140817, 0.024191717, 0.0020354858, 0.008200472, -0.014463005, -0.012453532, 0.0034596757, 0.0014843439, -0.0266369, 0.0069648735, -0.0044318964, -0.015620566, 0.0060154134, 0.018937172, -0.01588069, -0.0025882535, 0.03251575, 0.062118087, 0.017805625, -0.0098522715, -0.013968766, -0.016153824, 0.01641395, 0.0018794102, -0.019964669, 0.012290953, 0.024855038, -0.015334427, -0.012017821, 0.007647704, -0.01402079, -0.0008665447, 0.0069778794, 0.014007784, -0.009709203, -0.011022839, -0.017311385, -0.0050139283, -0.0033393675, 0.0025573636, 0.010463567, -0.028691897, -0.016752115, 0.008974346, 0.024373805, -0.003908393, -0.026142662, 0.015906705, -0.012440525, 0.006808798, 0.033998467, -0.006808798, 0.011764198, 0.0033913925, 0.003986431, 0.0067632757, 0.012154386, -0.025609404, 0.008616674, 0.0023037407, 0.028093606, -0.02762538, 0.0077127353, -0.015828667, 0.014033797, 0.009039378, 0.0032905939, 0.011829229, -0.020133752, -0.0066137034, 0.008902812, -0.020667009, 0.008655692, 0.020693023, 0.022266785, -0.01216089, 0.03620954, -0.0069973893, 0.0014347574, -0.0002489487, 0.020536946, 0.0008933702, -0.021135237, -0.0017704825, -0.007875314, -0.0022078194, 0.005384608, -0.030200627, -0.010821241, -0.0011136644, -0.013383482, 0.01986062, 0.007647704, 0.010353014, -0.0065454203, 0.0041490095, 0.005170004, -0.0070949364, -0.04031953, -0.0148401875, -0.0075826724, -0.020510934, 0.0012607982, 0.006425112, -0.017441448, 0.011165908, -0.0019606997, -0.017532492, -0.017649548, 0.019548468, -0.023593428, -0.027989557, 0.014944238, -0.007589176, 0.018039737, 0.014593068, -0.000074125746, 0.029342212, -0.0027589612, 0.00043571103, -0.017207334, 0.003755569, -0.008896309, -0.027677406, -0.0058365767, -0.010665165, 0.0009974206, 0.01662205, -0.0016526129, 0.005726023, -0.012193406, 0.013526551, -0.027183166, 0.032567773, -0.01203733, -0.004792821, 0.009540121, 0.006899842, -0.010918789, 0.022006659, 0.020953149, 0.0073745716, 0.011022839, -0.025505353, 0.018937172, 0.0181698, 0.013292438, -0.038706746, -0.0021818068, -0.025050133, 0.041984335, -0.004565211, 0.00012589691, -0.004968406, -0.004835092, -0.023788521, 0.008785755, 0.013578577, -0.012473041, 0.008473604, -0.01922331, 0.012499054, 0.016218856, -0.018338881, 0.024295768, -0.0015688848, 0.012687645, 0.037198015, -0.004158764, 0.023294283, -0.010918789, -0.01405981, -0.015412465, 0.0056187212, -0.010235958, 0.007114446, 0.01170567, 0.015048289, 0.027547343, -0.039461114, 0.034856882, 0.004243305, 0.006025168, 0.012941268, 0.02853582, 0.004786318, 0.015022276, -0.0034531725, 0.04073573, -0.027313229, -0.006386093, 0.0016266003, -0.034102518, -0.0015705107, -0.020836093, -0.0064218603, 0.007933843, -0.028769935, 0.009936812, 0.0038303551, 0.022682985, 0.020797072, 0.0044253934, 0.020406883, 0.012082852, -0.0028955275, 0.007972862, -0.0057032625, 0.002684175, 0.0074526095, -0.022344822, 0.0052838093, -0.004965155, -0.0088312775, 0.023333302, -0.009566133, 0.013266426, 0.0070168986, 0.008850787, -0.0021525426, -0.0006653535, 0.016192842, -0.0039409087, -0.049527988, 0.0058008097, -0.01205684, -0.010268473, -0.016518, -0.030954992, -0.0022159482, -0.038914848, 0.010730197, -0.0006629148, 0.012902249, -0.0037880847, 0.0011136644, -0.036079474, 0.010645656, 0.012785193, -0.0056122183, 0.007888321, -0.0036059965, -0.013435507, -0.004327846, -0.02762538, -0.0006722631, 0.02428276, 0.024724975, -0.007790773, 0.00027557096, 0.00685432, -0.025440322, 0.011634135, 0.011068361, -0.003983179, 0.020901123, 0.019210305, 0.011035845, -0.006737263, -0.01405981, 0.01205684, -0.000036885052, -0.011185418, -0.02287808, -0.010340008, -0.017805625, 0.10004445, -0.0103920335, -0.0067112506, 0.0101579195, -0.019587487, -0.023658458, -0.014996263, -0.0056252247, 0.003189795, 0.011022839, -0.009462083, -0.017155308, -0.010053869, 0.0041977833, 0.0023427596, 0.01216089, -0.004285576, -0.003175163, 0.0022809797, -0.006376338, 0.028431771, -0.023112195, 0.03280189, 0.025843518, 0.028509809, 0.0067437664, 0.000053498567, 0.009527114, 0.008259, -0.03293195, -0.0032401944, 0.0038108458, 0.0053553437, 0.013734652, -0.0080443965, -0.0177536, 0.014931232, 0.006051181, 0.034128528, -0.030408729, 0.015659584, 0.04341503, 0.00082183554, -0.01831287, 0.012421016, -0.024048647, -0.01655702, 0.01627088, -0.0214734, -0.025856523, 0.019821601, 0.01191377, -0.026337756, -0.030044552, 0.0096051525, 0.026090637, -0.00149166, 0.019275336, -0.0049586515, -0.011120386, -0.0010518845, -0.020549953, 0.020927135, 0.0032922195, 0.004054714, -0.020836093, 0.00082102267, -0.015763635, -0.0068478165, -0.0015916459, -0.014124841, -0.008870296, 0.010951304, -0.019678531, 0.0029410494, -0.008070408, 0.03092898, -0.008141943, 0.0028093606, 0.017129296, -0.02671494, -0.017766604, -0.028587846, -0.028249683, 0.010053869, 0.0061129606, 0.0047700605, -0.012128375, -0.011627631, 0.010424549, 0.020640997, 0.017831637, 0.024477856, 0.000978724, 0.00071412715, -0.014306929, 0.00279798, 0.012733167, 0.014645093, 0.013149369, -0.006447873, -0.033582266, 0.011465053, 0.00060560583, -0.0015802654, -0.01620585, 0.010795228, 0.021798559, -0.0015022276, 0.0061292187, 0.03553321, -0.01666107, 0.029056072, -0.003648267, 0.020393878, 0.021083212, -0.00060519937, 0.020016694, -0.00827851, -0.023034155, 0.023203239, -0.016296893, -0.017207334, 0.051504947, -0.010144914, -0.019392392, 0.020445902, -0.022982132, 0.0019330613, 0.01441098, -0.031189106, 0.02393159, -0.0031020024, -0.006073942, -0.025492348, -0.01866404, -0.01121143, 0.01567259, 0.01143904, -0.0037360594, -0.008941831, -0.00005959527, 0.018820114, -0.00044790443, 0.012791695, -0.021512419, 0.0009762854, 0.0043180916, -0.02091413, 0.007790773, -0.030876955, 0.006200753, 0.00017294314, -0.0009608404, -0.021356344, -0.031371195, -0.02256593, -0.025752474, 0.0216815, 0.016609045, 0.04312889, 0.0022078194, 0.031657334, 0.0011949538, -0.01013841, -0.0037490658, -0.019652518, -0.0024354295, -0.039513137, 0.01662205, 0.03784833, 0.006431615, 0.003113383, -0.010613141, -0.0014875955, 0.012551079, 0.0010844002, 0.0006515343, -0.004106739, -0.023775516, -0.0042042863, 0.0001352452, 0.008213478, -0.01108787, -0.017207334, -0.008480107, 0.02733924, 0.016426956, 0.013383482, -0.009045881, 0.017181322, -0.003466179, -0.005248042, -0.0060609356, 0.010593631, 0.011484562, -0.00082508713, -0.0064673824, 0.008473604, 0.0074005844, 0.015932716, -0.0062755393, -0.0064056027, -0.012648626, -0.011452046, 0.010626147, -0.008460598, -0.040163454, -0.00718598, -0.022708999, -0.010105895, -0.020940142, -0.008272006, -0.0464585, 0.005696759, 0.0025866278, -0.020640997, 0.024555894, 0.0016339164, -0.010704185, 0.021317326, -0.0064608795, 0.038758773, 0.005072457, 0.022513904, 0.0088312775, 0.009377542, -0.0055634445, 0.016960215, 0.014254904, -0.016426956, 0.004965155, 0.012876237, -0.000355641, -0.0117186755, 0.016817145, -0.00048367176, -0.034180555, -0.024490861, 0.021746533, 0.007836295, 0.016361924, -0.014749143, -0.012447028, -0.034128528, 0.02492007, -0.0076672137, 0.019249324, -0.024503868, -0.009546624, -0.028275695, 0.030200627, -0.01887214, 0.008746737, 0.015594553, -0.018911159, -0.0045847204, -0.008603667, -0.0030646094, 0.0016030264, 0.0022078194, 0.04083978, 0.016036768, 0.005995904, 0.011016335, 0.014788163, -0.0020094733, -0.0012624239, -0.012258437, 0.026740951, -0.01736341, -0.008746737, -0.0126811415, -0.0026906782, 0.0028126123, 0.011868248, -0.002944301, -0.006912848, -0.019743562, 0.0008844284, -0.00002194813, 0.023567414, -0.025674434, -0.002485829, -0.028015569, -0.022266785, 0.0009779112, -0.025570385, 0.009572636, -0.017324392, -0.030512778, -0.008245993, -0.024972094, 0.0067567728, -0.012590098, -0.022513904, 0.00848661, 0.04435148, -0.0326198, -0.0048318403, -0.031501256, 0.0071534645, -0.010359517, 0.01947043, -0.013032312, -0.032879926, 0.03423258, -0.019314354, -0.007972862, -0.022409854, 0.0067567728, -0.0072315023, -0.0031979238, 0.0042725694, 0.02133033, -0.013487533, -0.0018859134, -0.023593428, -0.0148401875, 0.00803139, 0.0037815815, -0.00084540946, 0.0021249042, -0.040111426, 0.0071924836, 0.010235958, 0.022630962, -0.009000359, -0.017272366, 0.010008347, -0.035377134, 0.010541606, -0.026506837, -0.01273967, -0.0088312775, 0.019873625, -0.012212915, 0.040111426, -0.008896309, 0.013981772, 0.022487892, 0.014319936, -0.0062170113, 0.01228445, -0.02140837, -0.0048448467, 0.0011030968, -0.006470634, -0.003449921, 0.004136003, -0.016400944, 0.013851709, 0.024061654, -0.020693023, -0.0083500445, 0.009364536, -0.040345542, 0.0068868357, -0.0042693177, 0.012830715, 0.0004094952, 0.009058887, -0.008811768, 0.008577654, 0.00048367176, -0.0020582469, -0.0035507197, 0.00032942518, 0.010424549, -0.0032954712, 0.0057065138, -0.0014046803, -0.018403914, -0.023905579, 0.021291312, -0.01275918, 0.021213274, 0.00069502415, -0.006951867, -0.011777204, -0.009819756, 0.00071981736, -0.0017298379, 0.011959292, 0.011881255, -0.039357062, -0.0025102159, -0.0062917974, -0.0142418975, -0.017194327, -0.008057402, 0.010444058, -0.0011526833, 0.013578577, -0.0041262484, -0.003625506, 0.0016648063, -0.028249683, 0.021421375, 0.0153214205, -0.0074526095, 0.019652518, 0.016465975, -0.004158764, -0.020354858, -0.022227766, -0.016244868, -0.019353373, -0.00365477, -0.015399459, 0.013201394, -0.029992526, 0.010372524, -0.013292438, 0.010457065, -0.0030824929, 0.017428441, -0.0009746596, 0.0005300067, 0.009299504, -0.028379746, -0.003012584, -0.002593131, -0.0071209488, -0.016114805, -0.009403555, -0.010502587, -0.001316075, 0.0075826724, 0.027781455, -0.005582954, 0.00104782, -0.021447388, 0.0058690924, -0.001032375, 0.01911926, 0.18895552, -0.006295049, -0.010587128, 0.047264893, -0.00097547245, 0.032437712, 0.035819348, -0.010769216, -0.013734652, 0.01588069, 0.0024500617, 0.00061251543, -0.009130422, -0.0040937327, -0.0014339446, -0.0073550623, -0.017805625, 0.0035734808, -0.0117186755, -0.019873625, -0.013272929, 0.00425306, -0.011640638, -0.0027296972, 0.008024887, 0.0060154134, 0.0070949364, 0.010040863, 0.006376338, -0.0058073127, -0.010522096, 0.012486047, 0.015932716, -0.019054228, -0.02502412, 0.0069973893, -0.01335747, -0.012492551, 0.010652159, 0.0032548264, -0.015503509, 0.008649189, 0.009598649, 0.0056122183, -0.009162938, 0.023788521, -0.011595116, 0.009988838, 0.008005377, 0.004841595, -0.02632475, -0.009735215, 0.029966515, 0.028093606, -0.0068933386, -0.0024972095, 0.0046465006, 0.0058333254, 0.017142303, 0.011243946, -0.0013030686, 0.020680016, -0.012102362, 0.028847972, -0.008226484, 0.0035051976, -0.01461908, 0.0073550623, 0.002163923, -0.016895182, -0.001193328, -0.017025245, 0.0024988353, -0.012843721, -0.022839062, -0.031475246, 0.025674434, 0.022370836, 0.011972299, 0.02382754, -0.011900764, -0.03204752, 0.00695837, -0.008915818, -0.01781863, 0.0044156387, -0.0011502446, 0.017688567, -0.013194891, 0.0021314074, 0.0012225922, -0.027001077, -0.00074583, 0.012271443, 0.010294486, 0.014710125, 0.01641395, 0.009540121, -0.008330535, 0.0005958511, -0.026662914, -0.0019444418, 0.024737982, 0.012147884, -0.0067892885, 0.016335912, -0.017558504, -0.0058170673, -0.0009689693, 0.0025866278, 0.0105676185, -0.02442583, -0.01170567, -0.005397614, -0.018078756, -0.019704543, 0.005498413, 0.016648063, 0.024490861, -0.013942753, 0.015867686, -0.002645156, 0.008805265, -0.007296534, 0.0083500445, -0.0025362284, -0.031553283, 0.0077517545, 0.0035832354, -0.041594144, 0.006704747, -0.011185418, 0.0037913362, -0.012011318, -0.025830511, 0.041802246, -0.01003436, 0.0097937435, 0.003641764, 0.022787036, -0.0017867404, -0.0026110145, 0.028145632, 0.011152902, 0.004311588, -0.021915615, -0.009182448, 0.02077106, -0.006171489, -0.0055309287, -0.019379387, 0.0022647218, -0.021213274, -0.014983257, 0.01158211, -0.0149572445, -0.021278305, -0.011725179, -0.021798559, 0.019691538, -0.04835742, 0.011536588, 0.029082086, -0.0006828307, -0.020003688, -0.0067957914, -0.16679278, 0.015646579, 0.037119977, -0.026480826, 0.0109317945, 0.0053488407, 0.013825696, -0.00035117008, -0.0013892354, -0.0014022416, 0.021915615, 0.008974346, -0.03274986, -0.0078688115, 0.023034155, 0.003518204, -0.017857648, 0.025037127, 0.0137606645, 0.02411368, 0.025765479, 0.0024338039, -0.00607069, -0.005114727, -0.009071894, -0.0094295675, 0.021096218, 0.016400944, -0.013175381, -0.0047440478, -0.015789647, -0.006548672, 0.017766604, 0.0077452515, 0.0044221417, 0.00838256, 0.0007568041, -0.015386452, -0.017649548, 0.014306929, -0.003066235, 0.02783348, -0.006984383, -0.02586953, 0.005108224, 0.02432178, 0.00885729, 0.0023297535, 0.007166471, -0.0021281557, -0.0052740546, -0.0035149525, 0.014085822, 0.00063893443, 0.031215118, -0.00425306, 0.014202879, -0.018351888, -0.0032158075, -0.0055081677, 0.003999437, -0.0017753599, 0.022748018, -0.023684472, -0.007946849, -0.019145273, -0.022487892, 0.018247837, -0.004535947, 0.008993856, -0.0043018335, 0.005498413, -0.0022647218, -0.023112195, 0.0055114194, 0.008818271, -0.00536835, 0.022604948, -0.020406883, -0.024386812, -0.014931232, 0.01761053, 0.0037588205, -0.0034986946, 0.01831287, 0.036001436, -0.000008446474, -0.024907064, -0.0036612733, -0.011725179, 0.023047162, -0.029758412, -0.014554049, -0.012934765, 0.015412465, 0.021850582, -0.011523581, -0.0022858572, -0.0040742233, -0.0056187212, 0.010307493, -0.019184291, 0.005192765, 0.008018384, 0.022722006, -0.0058138156, 0.03771827, 0.01028148, 0.029212149, -0.0054366332, 0.0014079319, -0.0009974206, 0.011627631, 0.026480826, 0.0025346025, 0.018937172, 0.0085971635, -0.012395004, 0.011608122, 0.00065397297, 0.040085416, 0.008883302, -0.0013339586, -0.0064413697, -0.007296534, 0.0028467537, -0.092917, -0.00801188, 0.016192842, 0.027469303, -0.033166062, 0.017662555, -0.002692304, 0.036079474, -0.0144760115, 0.020445902, -0.00036641184, -0.018442933, -0.008493113, -0.01476215, 0.018703058, 0.001552627, 0.0065876907, -0.017974706, -0.0076542073, 0.022149729, -0.010066876, 0.004932639, -0.0061292187, 0.0016436711, -0.007959855, 0.0031393955, -0.0324117, 0.037093967, 0.007114446, 0.019821601, -0.016518, -0.0034596757, 0.021668496, -0.03332214, 0.012785193, -0.007810283, -0.018833121, -0.0029963262, 0.0210572, -0.019392392, 0.0021135237, 0.015685597, 0.013116853, -0.027495317, 0.011250449, -0.0071924836, -0.019106254, 0.0107562095, 0.014007784, -0.034492705, -0.03394644, -0.024750987, -0.041047882, -0.006685238, 0.0068868357, 0.008694711, 0.019093247, 0.01261611, -0.009748221, -0.0027394518, 0.014918226, -0.0032710843, -0.0044936766, 0.01546449, 0.018208819, 0.004288827, -0.021538433, -0.026897028, 0.023879565, -0.018299863, -0.012245431, 0.015516515, -0.040891804, -0.011816223, -0.02221476, 0.012082852, -0.02417871, -0.009338523, 0.010047366, -0.027079115, -0.016010754, -0.021863589, -0.004305085, -0.027261203, 0.024100672, 0.000711282, 0.001071394, -0.0037263047, -0.007829792, -0.0049456456, -0.021603463, 0.03274986, 0.01866404, -0.032567773, -0.0044318964, 0.017935688, 0.018638028, -0.015282402, -0.009377542, 0.03012259, 0.015178352, -0.004792821, -0.048305396, 0.007075427, 0.020784067, 0.010678172, 0.0019509449, 0.0045684627, 0.02242286, -0.011374009, 0.024477856, -0.0021736778, -0.021291312, -0.009188951, 0.02692304, -0.0038628709, -0.02442583, -0.029394237, 0.019054228, 0.01606278, -0.0035051976, -0.00012284856, 0.01911926, 0.0088768, 0.007036408, 0.0015461239, -0.0030711126, 0.0034629272, -0.015152339, 0.02097916, -0.031163093, 0.0061909985, 0.012694148, -0.021902608, -0.030044552, 0.031553283, 0.0002003783, -0.037119977, 0.0003367412, 0.015347433, 0.003105254, 0.034102518, 0.001911926, -0.008102925, 0.0003306445, 0.011647142, -0.004288827, 0.019665524, -0.011907267, 0.01391674, 0.0014428863, -0.01655702, -0.0020387375, 0.009975832, -0.0009982334, -0.018586002, -0.0137606645, -0.031787395, 0.02481602, 0.0059308726, -0.0011941409, -0.009598649, 0.0126811415, 0.015828667, -0.0058918535, -0.0040807263, -0.00406772, -0.0015233628, -0.01402079, -0.0052577965, 0.0018290109, -0.018182807, -0.014736137, 0.014423986, -0.001820882, -0.013799684, 0.0029085337, 0.009949819, 0.02407466, 0.01992565, -0.01986062, 0.02892601, 0.0029768168, 0.00044018193, -0.012882739, 0.022487892, 0.0061682374, 0.014528036, 0.0021785551, 0.006304804, 0.0010722068, -0.0075696665, -0.0040417076, 0.0097937435, -0.01216089, -0.0044026324, -0.018494958, 0.053898104, -0.0037100469, -0.013929747, 0.0210572, 0.014293923, 0.028613858, -0.005791055, -0.008863793, -0.021668496, 0.0011803217, 0.0053781047, -0.020588972, -0.030304678, -0.015893698, -0.0041295, 0.04705679, 0.020198783, -0.007010395, 0.0039344057, 0.00019905735, 0.010587128, 0.0026549108, -0.015165345, -0.023996623, 0.01476215, -0.0021200269, 0.0458342, 0.02221476, -0.021174256, 0.009000359, 0.027573355, -0.007634698, -0.0053520924, -0.0032190592, 0.010196939, 0.014033797, 0.01203733, -0.032307647, -0.00873373, -0.016609045, -0.014593068, -0.014658099, 0.015828667, -0.024087666, 0.03043474, -0.0020598727, 0.006327565, 0.0019850864, -0.004236802, 0.015932716, 0.00049139425, -0.0023232503, -0.018351888, -0.025466334, -0.010457065, -0.00039587924, 0.0017330894, -0.024334786, -0.018599007, -0.0031442728, -0.030564804, 0.036807828, 0.0051895133, -0.015620566, 0.020680016, 0.021239286, 0.019171285, -0.00803139, -0.027963543, -0.0049261358, 0.018429926, 0.011217933, -0.015984742, -0.020575965, -0.007472119, -0.0071924836, -0.035975423, -0.028405758, 0.027573355, 0.0015770138, -0.009279994, -0.007634698, 0.007810283, 0.039929338, -0.029186135, 0.025245227, -0.02446485, -0.011770701, 0.009611655, -0.0033718832, -0.015451483, 0.007829792, -0.018403914], 
      'numCandidates': 200, 
      'limit': 10
    }
  }, {
    '$project': {
      '_id': 0, 
      'title': 1, 
      'genres': 1, 
      'plot': 1, 
      'year': 1, 
      'score': {
        '$meta': 'vectorSearchScore'
      }
    }
  }
]

# run pipeline
result = client["sample_mflix"]["embedded_movies"].aggregate(pipeline)

# print results
for i in result:
    print(i)
